#### Author : Jeonghun Yoon

Naive Bayes classifier를 이용하여 영화 리뷰를 예측하는 감정 분류기를 구현하라. 
 - 0 : 부정
 - 1 : 긍정

In [2]:
import pandas as pd

In [3]:
# 영화 리뷰를 load한다. 사랑/장르라는 단어를 포함하고 있는 document를 load 한다.
reviews = pd.read_csv('./inputs/ratings_train.txt', delimiter='\t')

In [4]:
# 데이터 확인
reviews.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [5]:
neg = reviews[(reviews.document.str.len() >= 30) & (reviews.label == 0)].sample(3000, random_state=43)
pos = reviews[(reviews.document.str.len() >= 30) & (reviews.label == 1)].sample(3000, random_state=43)

In [17]:
# 형태소 분석기
import re
import konlpy
from konlpy.tag import Twitter

okt = Twitter()

In [18]:
def parse(s):
    s = re.sub(r'[?$.!,-_\'\"(){}~]+', '', s)
    try:
        return okt.nouns(s)
    except:
        return []

In [19]:
neg['parsed_doc'] = neg.document.apply(parse)
pos['parsed_doc'] = pos.document.apply(parse)

In [20]:
neg.head()

,id,document,label,parsed_doc
149713,6674102,정말 댓글에 속아서 보게된 시간 아까운 영화. 전체적으로 코믹적인 요소를 많이 넣으...,0,"[정말, 댓글, 속, 시간, 영화, 전체, 코믹, 요소, 정말, 웃기, 스토리]"
52541,8514375,점수를 줘야 하는거임? 30년후 구구절절 설명하며 끝나는모습 이라니...,0,"[점수, 임, 년후, 구구, 절절, 모습]"
72078,1173600,주제는 좋으나 영화 자체는 주제를 포함하기엔 좀 조잡하다,0,"[주제, 영화, 자체, 주제, 좀, 조잡]"
19403,8384839,"이걸 영화라고 만들다니...말그대로 '돈만 떡으로 바르고 여운의 감동, 의미는 없는...",0,"[걸, 영화, 그대로, 돈, 떡, 여운, 감동, 의미, 영화, 이건, 그냥, 영화,..."
106111,8997490,"과장되고 어설픈 연기에 인류의 보편적 정서와는 거리가 먼 일본식 정서간의 괴리감, ...",0,"[과장, 어설픈, 연기, 인류, 보편, 정서, 거리, 일본, 정서, 괴리감, 무엇,..."


In [21]:
pos.head()

,id,document,label,parsed_doc
148005,8918849,마지막에 눈물이 흐를뻔... 너무 감동적이고 재미있는 영화,1,"[마지막, 눈물, 뻔, 너무, 감동, 영화]"
41010,9304354,오! 평점 높네요. 저도 진짜 좋아하는 영화에요. 소소한 일상의 반전. 아즈키판다 짱!,1,"[오, 평점, 저, 진짜, 영화, 일상, 반전, 아즈키판, 짱]"
97924,9324301,전 아프리카 구호 활동중 에볼라에 걸려 눈을 감는 순간 누군가 틀어준 이 영화를 보...,1,"[전, 아프리카, 구호, 활동, 에볼라, 눈, 순간, 누군가, 이, 영화, 보고, ..."
68271,10274621,전작을 본적이 없는데도 너무 재미있게 봤습니다. 크리스 에반스와 사무엘 L.잭슨의 ...,1,"[전작, 본적, 너무, 크리스, 에반스, 사무엘, 잭슨, 카리스마, 그, 외, 스토..."
41106,9908746,신선하고 환경파괴에대해서 자연과 인간의 공존 전쟁 등등 많은 교훈을 주는 애니메이션...,1,"[환경, 파괴, 자연, 인간, 공존, 전쟁, 등등, 교훈, 애니메이션, 원작, 만화..."


In [22]:
neg_train = neg[:2900]
pos_train = pos[:2900]
neg_test = neg[2900:]
pos_test = pos[2900:]

Corpus 생성하기

In [23]:
train_data = pd.concat([neg_train, pos_train], axis=0)
train_data.head(1)

,id,document,label,parsed_doc
149713,6674102,정말 댓글에 속아서 보게된 시간 아까운 영화. 전체적으로 코믹적인 요소를 많이 넣으...,0,"[정말, 댓글, 속, 시간, 영화, 전체, 코믹, 요소, 정말, 웃기, 스토리]"


In [24]:
corpus = train_data.groupby('label').agg({'parsed_doc':sum})
corpus

,parsed_doc
label,
0,"[정말, 댓글, 속, 시간, 영화, 전체, 코믹, 요소, 정말, 웃기, 스토리, 점..."
1,"[마지막, 눈물, 뻔, 너무, 감동, 영화, 오, 평점, 저, 진짜, 영화, 일상,..."


In [25]:
from collections import Counter

In [26]:
neg_corpus = Counter(corpus.parsed_doc.iloc[0]).most_common()
neg_corpus[:10]

[('영화', 1395),
 ('진짜', 305),
 ('너무', 298),
 ('스토리', 242),
 ('이', 234),
 ('평점', 206),
 ('정말', 199),
 ('감독', 195),
 ('점', 195),
 ('왜', 193)]

In [27]:
pos_corpus = Counter(corpus.parsed_doc.iloc[1]).most_common()
pos_corpus[:10]

[('영화', 1518),
 ('너무', 360),
 ('정말', 356),
 ('이', 270),
 ('연기', 247),
 ('최고', 244),
 ('것', 235),
 ('때', 200),
 ('진짜', 198),
 ('감동', 191)]

$$p(spam|doc) = \frac{p(doc|spam) \times p(spam)}{p(doc)} = \frac{\Pi_{i=1}^{n}p(word_i|spam) \times p(spam)}{p(doc)}$$

In [28]:
import numpy as np

`'tuple' object does not support item assignment``

In [29]:
neg_words, neg_cnts = zip(*neg_corpus)
neg_likelihoods = np.array(neg_cnts) / len(neg_train)
neg_likelihoods

array([4.81034483e-01, 1.05172414e-01, 1.02758621e-01, ...,
       3.44827586e-04, 3.44827586e-04, 3.44827586e-04])

In [30]:
pos_words, pos_cnts = zip(*pos_corpus)
pos_likelihoods = np.array(pos_cnts) / len(pos_train)
pos_likelihoods

array([5.23448276e-01, 1.24137931e-01, 1.22758621e-01, ...,
       3.44827586e-04, 3.44827586e-04, 3.44827586e-04])

In [31]:
neg_dicts = {}
for w, v in zip(neg_words, neg_likelihoods):
    neg_dicts[w] = v

In [32]:
neg_dicts

{'영화': 0.4810344827586207,
 '진짜': 0.10517241379310345,
 '너무': 0.10275862068965518,
 '스토리': 0.08344827586206896,
 '이': 0.08068965517241379,
 '평점': 0.07103448275862069,
 '정말': 0.06862068965517241,
 '감독': 0.06724137931034482,
 '점': 0.06724137931034482,
 '왜': 0.06655172413793103,
 '사람': 0.06551724137931035,
 '연기': 0.06413793103448276,
 '것': 0.06413793103448276,
 '뭐': 0.061724137931034484,
 '시간': 0.05862068965517241,
 '배우': 0.05620689655172414,
 '말': 0.05379310344827586,
 '쓰레기': 0.05241379310344828,
 '좀': 0.052068965517241377,
 '그냥': 0.05103448275862069,
 '내용': 0.04862068965517241,
 '더': 0.04758620689655172,
 '드라마': 0.04586206896551724,
 '이건': 0.04379310344827586,
 '거': 0.043448275862068966,
 '재미': 0.04310344827586207,
 '보고': 0.04275862068965517,
 '하나': 0.041379310344827586,
 '돈': 0.04068965517241379,
 '내': 0.03896551724137931,
 '때': 0.03793103448275862,
 '걸': 0.037586206896551726,
 '그': 0.03586206896551724,
 '최악': 0.035517241379310345,
 '나': 0.03482758620689655,
 '생각': 0.032413793103448274

In [33]:
pos_dicts = {}
for w, v in zip(pos_words, pos_likelihoods):
    pos_dicts[w] = v

In [34]:
pos_dicts

{'영화': 0.523448275862069,
 '너무': 0.12413793103448276,
 '정말': 0.12275862068965518,
 '이': 0.09310344827586207,
 '연기': 0.08517241379310345,
 '최고': 0.08413793103448276,
 '것': 0.08103448275862069,
 '때': 0.06896551724137931,
 '진짜': 0.06827586206896552,
 '감동': 0.06586206896551725,
 '보고': 0.0596551724137931,
 '드라마': 0.05931034482758621,
 '사람': 0.05827586206896552,
 '더': 0.05551724137931034,
 '봐': 0.05344827586206897,
 '배우': 0.052068965517241377,
 '수': 0.05137931034482759,
 '점': 0.05068965517241379,
 '그': 0.0503448275862069,
 '평점': 0.04931034482758621,
 '스토리': 0.04862068965517241,
 '마지막': 0.047241379310344826,
 '다시': 0.04448275862068966,
 '내': 0.04275862068965517,
 '지금': 0.04206896551724138,
 '작품': 0.039310344827586205,
 '생각': 0.038275862068965515,
 '사랑': 0.03517241379310345,
 '나': 0.03379310344827586,
 '왜': 0.03310344827586207,
 '이야기': 0.032758620689655175,
 '장면': 0.031724137931034485,
 '재미': 0.031379310344827584,
 '내용': 0.028965517241379312,
 '거': 0.028620689655172414,
 '하나': 0.02793103448275

# Test

In [35]:
test_data = pd.concat([neg_test, pos_test], axis=0)

In [36]:
def predict(doc):
    pos_prior, neg_prior = 1/2, 1/2

    # Posterior of pos
    pos_prob = 1
    for word in doc:
        if word in pos_dicts:
            pos_prob *= (pos_dicts[word] * len(pos_train) + 1) / (len(pos_train) + len(pos_dicts))
        else:
            pos_prob *= 1 / len(pos_dicts)
    pos_prob *= pos_prior
    
    # Posterior of neg
    neg_prob = 1
    for word in doc:
        if word in neg_dicts:
            neg_prob *= (neg_dicts[word] * len(neg_train) + 1) / (len(neg_train) + len(neg_dicts))
        else:
            neg_prob *= 1 / len(neg_dicts)
    neg_prob *= neg_prior
    
    print(pos_prob)
    print(neg_prob)
    
    if pos_prob >= neg_prob:
        return 1
    else:
        return 0

In [37]:
test_data['pred'] = test_data.parsed_doc.apply(predict)

1.4603213282216426e-12
1.5097731762231753e-12
5.593532023926221e-20
2.7138725664141513e-18
9.466758646596474e-52
4.78701777025903e-51
6.26386090167354e-16
3.1554186277451635e-16
2.0306995246916366e-64
2.2832661064626267e-67
2.35666807442678e-78
3.129597544571833e-73
2.1423269446049754e-11
1.8229013660495034e-11
5.0978724071505846e-39
2.6355254549633755e-35
3.1751721212705667e-37
3.483603987897944e-36
2.8480178894975193e-40
3.279675269505989e-39
1.2855995010767263e-13
7.743905352034712e-13
4.04896995057731e-28
1.0282225768297874e-25
2.1520186608990834e-43
3.471613169533957e-40
1.2282880850781385e-25
3.784448931776959e-22
1.3378874843199421e-18
4.424233439820444e-18
2.5816235885648723e-19
8.366723377418374e-19
7.194767061885266e-29
1.532574735721924e-27
2.9295924116274104e-32
1.0928394113644318e-31
2.4368968994881592e-09
4.712200031237967e-09
9.920962218177029e-51
3.5586534754468626e-46
2.520566160264892e-11
7.20837127524511e-11
7.486784852734175e-14
1.6785382875022142e-13
2.395101804536

In [38]:
test_data[:10]

,id,document,label,parsed_doc,pred
91417,5079332,쉴새없이 떠드는 폴도 싫지만...영화를 이렇게 만든 감독은 더 싫다.,0,"[새, 폴, 영화, 감독, 더]",0
122641,2776735,별표 반개도 아깝다..2시간 동안 본 짜증이 확밀려오네..,0,"[별표, 반개, 시간, 동안, 짜증, 확]",0
72606,8477996,내용은 봐줄만해. 하지만 15금과 19금 사이에 서있다. 포스터는 호러 귀신나올것같...,0,"[내용, 만해, 금, 금, 사이, 포스터, 호러, 귀신, 막상, 차마, 두, 최악,...",0
138243,7923759,진정한 수면제...졸다가 비명지르는 소리에 깼다가 다시 자다,0,"[진정, 수면제, 비명, 소리, 다시]",1
118247,8142893,"""진짜 슈스케 버스커 버스커, 허각, 울랄라세션 나올땐 재밌는데 갈수록 재미가 없어...",0,"[진짜, 슈스케, 버스커, 버스커, 허각, 울랄라세션, 땐, 갈수록, 재미, 제, ...",1
63089,8525344,내가씨x 아침에 티비에서 틀어주는거 보고 욕이 나와서 평점쓴다. 개그맨 성우새.끼들...,0,"[가씨, 아침, 티비, 보고, 욕, 평점, 개그맨, 성우, 새끼, 조화, 는걸, 영...",0
132881,1872310,이송희일 : 관심받고싶어서 디워까댄거에요 죄송해요 ㅠㅠ,0,"[이송희일, 관심, 디워]",1
140268,4712973,1등!! 이건 무슨 병맛 출연진에 뭐냐 이거 긴급조치 19호 스맬난다...,0,"[등, 이건, 무슨, 병맛, 출연, 진, 뭐, 거, 긴급, 조치, 호, 스, 난]",0
26536,5799940,"모모는 철부지,모모는 무지개,모모는 생을 쫒아가는 시계바늘이다. 이노래모르면?간첩",0,"[모모, 철부지, 모모, 무지개, 모모, 생, 시계, 바늘, 노래, 간첩]",0
9821,8951429,진짜한편봤는데. 정말 다들 무례하네요.음식의 평가를 제대로하는게아니라 깎아내리기 바...,0,"[진짜, 한편, 정말, 무례, 음식, 평가, 제대로, 리기, 프로그램, 취지, 황은...",0


In [39]:
sum(test_data.label ^ test_data.pred)

39

In [41]:
1 - 39/200

0.8049999999999999

# TF-IDF

$$TF=\frac{\text{term frequnecy of the word in the document}}{\text{the number of words in the documnet}}$$

In [47]:
tf_vals = np.array(neg_cnts) / sum(neg_cnts)
tf_vals

array([4.51690196e-02, 9.87566377e-03, 9.64900920e-03, ...,
       3.23792255e-05, 3.23792255e-05, 3.23792255e-05])

In [46]:
neg_tf = {}
for w, v in zip(neg_words, tf_vals):
    neg_tf[w] = v

In [50]:
neg_tf

{'영화': 0.045169019557052194,
 '진짜': 0.009875663774122523,
 '너무': 0.00964900919570004,
 '스토리': 0.007835772568320166,
 '이': 0.0075767387644087554,
 '평점': 0.006670120450718819,
 '정말': 0.006443465872296335,
 '감독': 0.00631394897034063,
 '점': 0.00631394897034063,
 '왜': 0.006249190519362777,
 '사람': 0.006152052842895998,
 '연기': 0.0060225359409402926,
 '것': 0.0060225359409402926,
 '뭐': 0.005795881362517808,
 '시간': 0.005504468333117472,
 '배우': 0.005277813754694988,
 '말': 0.005051159176272504,
 '쓰레기': 0.004921642274316798,
 '좀': 0.004889263048827872,
 '그냥': 0.004792125372361093,
 '내용': 0.004565470793938609,
 '더': 0.00446833311747183,
 '드라마': 0.0043064369900271985,
 '이건': 0.0041121616370936405,
 '거': 0.004079782411604714,
 '재미': 0.004047403186115788,
 '보고': 0.004015023960626861,
 '하나': 0.0038855070586711568,
 '돈': 0.0038207486076933038,
 '내': 0.0036588524802486726,
 '때': 0.0035617148037818936,
 '걸': 0.003529335578292967,
 '그': 0.0033674394508483355,
 '최악': 0.0033350602253594095,
 '나': 0.0032703017

In [48]:
tf_vals = np.array(pos_cnts) / sum(pos_cnts)
tf_vals

array([5.34638819e-02, 1.26791815e-02, 1.25383017e-02, ...,
       3.52199486e-05, 3.52199486e-05, 3.52199486e-05])

In [49]:
pos_tf = {}
for w, v in zip(pos_words, tf_vals):
    pos_tf[w] = v

In [51]:
pos_tf

{'영화': 0.05346388194273236,
 '너무': 0.012679181488395027,
 '정말': 0.012538301694079526,
 '이': 0.00950938611629627,
 '연기': 0.008699327298982144,
 '최고': 0.008593667453245519,
 '것': 0.008276687916035643,
 '때': 0.007043989715775015,
 '진짜': 0.006973549818617265,
 '감동': 0.0067270101785651395,
 '보고': 0.006093051104145388,
 '드라마': 0.006057831155566513,
 '사람': 0.0059521713098298875,
 '더': 0.005670411721198887,
 '봐': 0.0054590920297256365,
 '배우': 0.005318212235410136,
 '수': 0.0052477723382523865,
 '점': 0.005177332441094636,
 '그': 0.005142112492515761,
 '평점': 0.005036452646779136,
 '스토리': 0.0049660127496213855,
 '마지막': 0.004825132955305885,
 '다시': 0.004543373366674885,
 '내': 0.0043672736237805095,
 '지금': 0.004296833726622759,
 '작품': 0.004015074137991758,
 '생각': 0.003909414292255133,
 '사랑': 0.0035924347550452575,
 '나': 0.0034515549607297574,
 '왜': 0.003381115063572007,
 '이야기': 0.003345895114993132,
 '장면': 0.003240235269256507,
 '재미': 0.003205015320677632,
 '내용': 0.0029584756806255064,
 '거': 0.002923

$$IDF = \text{log}_e{\frac{\text{the total number of documents}}{\text{the number of documents contains the word}}}$$

In [66]:
neg_train.parsed_doc.iloc[0]

['정말', '댓글', '속', '시간', '영화', '전체', '코믹', '요소', '정말', '웃기', '스토리']

In [77]:
idfs = []
for word in pos_words:
    cnt = 0
    for _, doc in pos_train.parsed_doc.items():
        if word in doc:
            cnt += 1
    idfs.append(np.log(len(pos_train) / cnt))

In [78]:
pos_idf = {}
for w, v in zip(pos_words, idfs):
    pos_idf[w] = v

In [82]:
pos_idf

{'영화': 0.9319296257586098,
 '너무': 2.245618268387368,
 '정말': 2.207274913189721,
 '이': 2.4752977906813634,
 '연기': 2.578838469622204,
 '최고': 2.5743033144568126,
 '것': 2.6691611079154898,
 '때': 2.7521101908962406,
 '진짜': 2.7467193422613643,
 '감동': 2.77396898470874,
 '보고': 2.8848696807421814,
 '드라마': 3.0524850901464404,
 '사람': 2.995732273553991,
 '더': 2.955186179159641,
 '봐': 3.030823593365261,
 '배우': 3.023706125596397,
 '수': 3.045212330817361,
 '점': 3.23626756758007,
 '그': 3.104931565518983,
 '평점': 3.104931565518983,
 '스토리': 3.045212330817361,
 '마지막': 3.104931565518983,
 '다시': 3.193342522863036,
 '내': 3.2275338876113153,
 '지금': 3.193342522863036,
 '작품': 3.3474932026902944,
 '생각': 3.2996371815126593,
 '사랑': 3.450677438925525,
 '나': 3.5181187197210577,
 '왜': 3.4291712337045617,
 '이야기': 3.4838296462424254,
 '장면': 3.6417326756882344,
 '재미': 3.495129201496359,
 '내용': 3.603018163507544,
 '거': 3.578016861302127,
 '하나': 3.7980787460789283,
 '말': 3.6286605941208814,
 '처음': 3.6417326756882344,
 '볼':

In [80]:
idfs = []
for word in neg_words:
    cnt = 0
    for _, doc in neg_train.parsed_doc.items():
        if word in doc:
            cnt += 1
    idfs.append(np.log(len(neg_train) / cnt))

In [81]:
neg_idf = {}
for w, v in zip(neg_words, idfs):
    neg_idf[w] = v

In [83]:
pos_tf_idf = {}
for w, tf in pos_tf.items():
    pos_tf_idf[w] = tf * pos_idf[w]

In [84]:
neg_tf_idf = {}
for w, tf in neg_tf.items():
    neg_tf_idf[w] = tf * neg_idf[w]

In [85]:
pos_tf_idf

{'영화': 0.049824575490493066,
 '너무': 0.028472601578538814,
 '정말': 0.02767547878334592,
 '이': 0.023538562444404186,
 '연기': 0.022434159898449775,
 '최고': 0.022122706608229573,
 '것': 0.022091813487836443,
 '때': 0.019385835881352734,
 '진짜': 0.01915438417101927,
 '감동': 0.018660517595159697,
 '보고': 0.0175776583935617,
 '드라마': 0.018491439280991364,
 '사람': 0.017831111690579524,
 '더': 0.016757122348631786,
 '봐': 0.01654554492204471,
 '배우': 0.016080710913431336,
 '수': 0.015980581033768423,
 '점': 0.016755233065694725,
 '그': 0.01596590739146168,
 '평점': 0.015637840801226165,
 '스토리': 0.01512256326014327,
 '마지막': 0.014981707620755137,
 '다시': 0.014508547369046302,
 '내': 0.014095523617222664,
 '지금': 0.013721261852896502,
 '작품': 0.013440433385225003,
 '생각': 0.012899648756662035,
 '사랑': 0.012396333560046616,
 '나': 0.01214298011948944,
 '왜': 0.011594422513846297,
 '이야기': 0.011656528594830783,
 '장면': 0.011800070656968887,
 '재미': 0.011201942638543608,
 '내용': 0.010659441613589043,
 '거': 0.010459458299160939,
 

In [86]:
neg_tf_idf

{'영화': 0.045374846385535884,
 '진짜': 0.024048542398380675,
 '너무': 0.02283596221358028,
 '스토리': 0.019927312966548056,
 '이': 0.019504823713990874,
 '평점': 0.01832094885720247,
 '정말': 0.017768052111603108,
 '감독': 0.01734264576288583,
 '점': 0.01798529650082259,
 '왜': 0.017617558067122775,
 '사람': 0.01756086383486543,
 '연기': 0.01726372299008422,
 '것': 0.017374231337198735,
 '뭐': 0.016939100879477403,
 '시간': 0.01601729267909247,
 '배우': 0.015527476344284651,
 '말': 0.015309172405290143,
 '쓰레기': 0.014881599892845885,
 '좀': 0.014853542433280175,
 '그냥': 0.014593039274736737,
 '내용': 0.014003172014395646,
 '더': 0.013805648390350046,
 '드라마': 0.014290935550889347,
 '이건': 0.013201239940080777,
 '거': 0.01302814265900604,
 '재미': 0.012857286420063075,
 '보고': 0.012889399626535562,
 '하나': 0.012678160139861021,
 '돈': 0.01253632754118507,
 '내': 0.012038849603033579,
 '때': 0.011922816095581285,
 '걸': 0.011779994043006,
 '그': 0.01130565768390959,
 '최악': 0.01143649258747474,
 '나': 0.011111684327955881,
 '생각': 0.01

In [87]:
def predict(doc):
    pos_prior, neg_prior = 1/2, 1/2

    # Posterior of pos
    pos_prob = 1
    for word in doc:
        if word in pos_dicts:
            pos_prob *= pos_tf_idf[word]
        else:
            pos_prob *= 1 / len(pos_dicts)
    pos_prob *= pos_prior
    
    # Posterior of neg
    neg_prob = 1
    for word in doc:
        if word in neg_dicts:
            neg_prob *= neg_tf_idf[word]
        else:
            neg_prob *= 1 / len(neg_dicts)
    neg_prob *= neg_prior
    
    print(pos_prob)
    print(neg_prob)
    
    if pos_prob >= neg_prob:
        return 1
    else:
        return 0

In [88]:
test_data['pred'] = test_data.parsed_doc.apply(predict)

1.2123242228431983e-12
6.733878390304011e-13
3.345498751875356e-19
9.332161731988884e-18
1.3914074431661167e-48
1.3879329870549335e-48
1.6348592942921865e-15
1.733040254300404e-15
1.0921523772122292e-61
6.451987270850159e-65
4.649023106319173e-75
2.5337620444387854e-70
6.486214443064632e-11
4.99812076830935e-11
5.04754754570154e-37
6.45331794332712e-34
8.256765750882915e-37
8.994272235182657e-35
4.647779811199318e-39
6.966353836241074e-38
9.3176643767442e-13
4.802420661918756e-12
7.408248373299441e-28
5.813993924965574e-26
1.9010905191783072e-42
2.425498303427318e-39
3.526965626777328e-24
3.464883199902345e-21
5.783813284734483e-18
8.920116383051338e-18
1.0457688947146604e-18
1.9183372679155114e-18
1.0010095533043317e-27
7.415714673051157e-27
1.2697017487474324e-31
6.2099178748656515e-31
4.173586231083988e-09
6.162734166323054e-09
3.872331645234292e-49
1.9102504505206784e-45
8.25995361620698e-11
2.1273798106108796e-10
4.310232467113627e-13
4.062850058059383e-13
6.388405804777683e-19
2.

In [93]:
test_data.head()

,id,document,label,parsed_doc,pred
91417,5079332,쉴새없이 떠드는 폴도 싫지만...영화를 이렇게 만든 감독은 더 싫다.,0,"[새, 폴, 영화, 감독, 더]",1
122641,2776735,별표 반개도 아깝다..2시간 동안 본 짜증이 확밀려오네..,0,"[별표, 반개, 시간, 동안, 짜증, 확]",0
72606,8477996,내용은 봐줄만해. 하지만 15금과 19금 사이에 서있다. 포스터는 호러 귀신나올것같...,0,"[내용, 만해, 금, 금, 사이, 포스터, 호러, 귀신, 막상, 차마, 두, 최악,...",1
138243,7923759,진정한 수면제...졸다가 비명지르는 소리에 깼다가 다시 자다,0,"[진정, 수면제, 비명, 소리, 다시]",0
118247,8142893,"""진짜 슈스케 버스커 버스커, 허각, 울랄라세션 나올땐 재밌는데 갈수록 재미가 없어...",0,"[진짜, 슈스케, 버스커, 버스커, 허각, 울랄라세션, 땐, 갈수록, 재미, 제, ...",1


In [91]:
sum(test_data.label ^ test_data.pred)

41

In [95]:
1 - 41/len(test_data)

0.795

In [96]:
len(test_data)

200